In [2]:
import IPython.display as ipd
import re
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
import pandas.api.types as at
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from catboost import Pool, CatBoostRegressor, cv

In [4]:
train = pd.read_csv('train150121.csv', low_memory=False) 
sample_submission = pd.read_csv('sample_submission.csv')
data = pd.read_csv('data_datasetcar.csv', low_memory=False) 
data.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
data

,brand,color,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,productionDate,...,Владельцы,ПТС,Привод,Руль,bodyType2,bT_length,Nalog,years_old,age_intervals,sample
0,8,14,1.2,105.0,0,74000,2013,710,5,2014,...,2,1,1,0,4,7,2625.0,7,1,0
1,8,17,1.6,110.0,0,60563,2017,710,5,2017,...,0,1,1,0,4,7,2750.0,4,1,0
2,8,13,1.8,152.0,0,88000,2013,954,5,2014,...,0,1,1,0,4,7,6840.0,7,1,0
3,8,7,1.6,110.0,0,95000,2013,710,5,2014,...,0,1,1,0,4,7,2750.0,7,1,0
4,8,1,1.8,152.0,0,58536,2008,710,5,2012,...,0,1,1,0,4,7,6840.0,9,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224131,9,17,2.5,179.0,0,152000,2012,818,5,2013,...,1,1,2,0,1,17,8950.0,8,2,1
224132,9,17,2.0,148.0,0,115000,2010,818,5,2011,...,1,1,2,0,1,17,5180.0,10,2,1
224133,9,17,2.0,146.0,0,258000,2012,818,5,2013,...,0,1,1,0,1,17,5110.0,8,2,1
224134,9,17,2.0,148.0,0,147000,2010,818,5,2011,...,0,1,2,0,1,17,5180.0,10,2,1


In [5]:
X = data.query('sample == 1').drop(['sample'], axis=1)
y = train['price']
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)

In [9]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [8]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [10]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 27.2min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 64.6min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 119.3min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed: 162.1min finished


{'bootstrap': True,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 100}

In [24]:
rf = RandomForestRegressor(bootstrap= True,
  max_depth = 80,
 min_samples_leaf = 3,
 min_samples_split = 8,
 n_estimators = 100)
rf.fit(X_train,np.log(y_train))


RandomForestRegressor(max_depth=80, max_features=3, min_samples_leaf=3,
                      min_samples_split=8)

In [26]:
y_rf = np.exp(rf.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, y_rf))*100:0.2f}%")

Точность модели по метрике MAPE: 12.15%


In [14]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 100}

In [11]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)

Model Performance
Average Error: 140860.7098 degrees.
Accuracy = 86.41%.


In [13]:
y_pred = best_grid.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

Точность модели по метрике MAPE: 13.59%


In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
rf = RandomForestRegressor()
log_rf = TransformedTargetRegressor(rf, func = np.log, inverse_func=np.exp)
gscv = GridSearchCV(log_rf, param_grid=param_grid,refit = True)
gscv.fit(Xtrain, ytrain)
gscv.best_params_

In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_regression
import numpy as np

rf = RandomForestRegressor(bootstrap= True,
  max_depth = 80,
 min_samples_leaf = 3,
 min_samples_split = 8,
 n_estimators = 1000)
log_rf = TransformedTargetRegressor(rf, func = np.log, inverse_func=np.exp)
#params = {'regressor__n_estimators': [10,100,1000]}
#gscv = GridSearchCV(log_rf, param_grid=params,refit = True)
#gscv.best_params_
log_rf.fit(X_train, y_train)

TransformedTargetRegressor(func=<ufunc 'log'>, inverse_func=<ufunc 'exp'>,
                           regressor=RandomForestRegressor(max_depth=80,
                                                           min_samples_leaf=3,
                                                           min_samples_split=8,
                                                           n_estimators=1000))

In [45]:
y_pred = log_rf.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

Точность модели по метрике MAPE: 11.41%


In [46]:
predict_submission = log_rf.predict(X_sub)
predict_submission

array([ 631454.8851736 ,  985906.1719744 ,  872578.9746519 , ...,
        247391.22261925, 1282721.52391125, 1097020.9045735 ])

In [49]:
#submission = pd.read_csv('sample_submission.csv')
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_log_rf.csv', index=False)
sample_submission.head(10)

,sell_id,price
0,1100575026,6.314549e+05
1,1100549428,9.859062e+05
2,1100658222,8.725790e+05
3,1100937408,7.489572e+05
4,1101037972,7.663281e+05
5,1100912634,7.444849e+05
6,1101228730,6.212512e+05
7,1100165896,3.708574e+05
8,1100768262,2.110528e+06
9,1101218501,8.028159e+05


In [ ]:
rf = RandomForestRegressor()
log_rf = TransformedTargetRegressor(rf, func = np.log, inverse_func=np.exp)

In [10]:
grid_search.fit(X_train, np.log(y_train))
grid_search.best_params_
{'bootstrap': True,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 5,
 'min_samples_split': 12,
 'n_estimators': 100}
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, test_features, test_labels)

NameError: name 'grid_search' is not defined

In [10]:
y_pred = gscv.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

Точность модели по метрике MAPE: 11.02%
